In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# for문으로 크롤링 

In [26]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

# import chromedriver_autoinstaller

# chrome_path = chromedriver_autoinstaller.install()
# driver = webdriver.Chrome(chrome_path, options=options)
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

In [27]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [28]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [70]:
# 우선 수기로 옵션 누르고 조회 기간을 설정해줌

In [ ]:
# 뉴스 데이터 수집
news_list = []
date_list = []
title_list = []
MAX_ITERATION = 350

for _ in range(120):
    time.sleep(1)

    # 기간 변경해 주기
    if len(date_list):
        splited_date = date_list[-1].split('.')
        year = int(splited_date[0]) - 1990 + 1
        month = int(splited_date[1])
        day = int(splited_date[2])
        

        # 기간을 클릭 해서 입력창 활성화  
        time.sleep(4)   
        driver.find_element(by=By.CSS_SELECTOR, value='#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger').click()
        time.sleep(2)

        ## 시작 일자 조정 ##
        
        # 년도 
        # 2016년 CSS_SELECTOR : snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(1) > div > div > div > ul > li:nth-child(27)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(1) > div > div > div > ul > li:nth-child({year})').click()
        time.sleep(1)
        
        # 월
        # 1월 CSS_SELECTOR : snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child(1)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child({month})').click()
        time.sleep(1)
        
        # 일
        # 1일 CSS_SELECTOR : #snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child(1)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child({day})').click()
        time.sleep(1)
        
        # "적용" 버튼 클릭
        driver.find_element(by=By.CSS_SELECTOR, value='#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.btn_area > button').click()
        time.sleep(2)
        
    for i in range(MAX_ITERATION):

        # 언론사
        newses = driver.find_elements('css selector', 'a.info.press')
        temp_list = []
        for news in newses:
            temp_list.append(news.text)
            temp_list = [x for x in temp_list ]
        news_list = news_list + temp_list 
        
        # 날짜
        dates = driver.find_elements('css selector', 'span.info')
        temp_list = []
        for date in dates:
            temp_list.append(date.text)
            temp_list = [x for x in temp_list ]
        date_list = date_list + temp_list 
        
        # 제목
        titles = driver.find_elements('css selector', 'a.news_tit')
        temp_list = []
        for title in titles:
            temp_list.append(title.text)
            temp_list = [x for x in temp_list ]
        title_list = title_list + temp_list 
        
        # 다음페이지 클릭
        driver.find_element('css selector', 'a.btn_next').click()
        time.sleep(0.001)

In [19]:
print(len(title_list))
print(news_list[-2:])
print(date_list[-2:])
print(title_list[-2:])

100
['연합뉴스', '연합뉴스']
['1990.04.23.', '1990.04.23.']
['여신관리', '蘇, 北韓에 원유공급량 줄여']


In [20]:
# df로 만들기
df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])
df

,언론사,날짜,제목
0,연합뉴스,1990.01.17.,朴守敎-林正明 코트서 벤치 대결로
1,연합뉴스,1990.01.17.,"EC, 한국산 CDP 반덤핑 확정관세 결정"
2,연합뉴스,1990.01.17.,범용 CAD 개발
3,연합뉴스,1990.01.19.,"광고업계,제조업 불황속 호황 누려"
4,연합뉴스,1990.01.21.,-농구대잔치-
...,...,...,...
95,연합뉴스,1990.04.20.,과학의 날 기념식 21일 大田에서
96,연합뉴스,1990.04.21.,官需用 복사기 경쟁입찰 구매 1년 연기
97,연합뉴스,1990.04.22.,등산하던 20대 회사원 떨어져 숨져
98,연합뉴스,1990.04.23.,여신관리


In [ ]:
# csv로 뽑기
df.to_csv('2008.csv')

In [100]:
driver.close()